Последняя лабораторная будет посвящена GPT и основана вот на этой тетрадке:
https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/gpt/gpt.ipynb
Напоминаю, что эта лаба необязательная!

1. %pip install transformers

2. загружаем Сберовскую модель.

3. берем любое предложение из Толстого (в тетрадке это пример про дождь, но можно подлиннее). 

4. пытаемся генерировать текст.

5. подбираем параметры, при которых генерированный текст будет длиной не менее 50 токенов и будет наиболее семантически и грамматически верным.

6. сдаем мне текст с описанием параметров.

# 1. Install transformers

In [2]:
!pip install transformers


Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: python-apt 2.4.0-elementary9-ubuntu7.1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-apt or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


# 2. Загрузка сберовской модели

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


In [3]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1536)
    (wpe): Embedding(2048, 1536)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1536, out_features=50257, bias=False)
)

# 3. Берем любое предложение из Толстого (в тетрадке это пример про дождь, но можно подлиннее)

In [4]:
input_ids = tokenizer.encode("Дождь шел не переставая с самого утра", return_tensors="pt")
max_length = 100

outputs = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Дождь шел не переставая с самого утра.

— Я не могу, — сказал он. — Я не могу.

— Почему?

— Потому что я не могу.

— Почему?

— Потому что я не могу.

— Почему?

— Потому что я не могу.

— Почему?

— Потому что я не могу.

— Почему?




Чисто я и научник в мае. Классика.

![sunukun](./sunukun.jpg)

Ладно, для начала посмотрим, че можно изменить в принципе, используя разные возможности, параметры...

В ноуте есть много всего, пройдемся, чтобы не открывать вкладку лишний раз:

Prompt engeniering (english is very well):

In [23]:
#!pip install torch torchvision torchaudio

In [30]:
# import gc
# torch.cuda.empty_cache()
# gc.collect()

# import torch 
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)


# input_text = "Дождь шел не переставая с самого утра"
# input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
# max_length = 100

# torch.backends.cudnn.benchmark = True
# model.config.gradient_checkpointing = True
# outputs = model.generate(input_ids, max_length=max_length, num_return_sequences=1, batch_size=1,  use_cache=False)
# generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print(generated_text)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 1.96 GiB total capacity; 1.21 GiB already allocated; 10.56 MiB free; 1.23 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

Я попытался выделить побольше памяти, сборщик даже включил, но это не отменяет, что я бомж с 2 гб памяти на видяхе. Штош.

In [6]:
# text = "Вопрос: 'Сколько будет 2+2?' \n Ответ: " # работает
text = " Вопрос: 'Сколько будет 3+3?' \n Ответ: 6 . \n Вопрос: 'Сколько будет 1+9?' \n Ответ: 10 . \n Вопрос: 'Сколько будет 4+2?' \n Ответ:"
input_ids = tokenizer.encode(text, return_tensors="pt")

max_length = len(input_ids[0]) + 50  # например, текущая длина + 50 токенов

out = model.generate(input_ids, max_length=max_length, do_sample=False)

generated_text = tokenizer.decode(out[0], skip_special_tokens=True)
print(generated_text)

 Вопрос: 'Сколько будет 3+3?' 
 Ответ: 6. 
 Вопрос: 'Сколько будет 1+9?' 
 Ответ: 10. 
 Вопрос: 'Сколько будет 4+2?' 
 Ответ: 12. 
 Вопрос: 'Сколько будет 5+3?' 
 Ответ: 13. 
 Вопрос: 'Сколько будет 6+3?' 
 Ответ: 14. 
 Вопрос: 'Сколько будет 7+3?' 
 Ответ: 15.


Мягко говоря, все очень плохо, мда. Попробуем просто повторить то, что делали умные люди до меня.

In [7]:
# возьмем какой-нибудь текст
text = 'За окном дождь. Холодный и противный. Хочется'
input_ids = tokenizer.encode(text, return_tensors="pt")

In [9]:
# argmax
# если повторить запуск результат не изменится
# но тут уже нет зацикливания потому что модель смотрит дальше чем два токена в прошлое
out = model.generate(input_ids, do_sample=False, max_length=50)


generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


За окном дождь. Холодный и противный. Хочется плакать.

— Я не могу, — говорит она. — Я не могу.

— Почему?

— Потому что я не могу.




Стоит мужик у букмекерского окна на ипподроме и думает, на какую бы лошадь поставить. Тут подходит к нему старая кобыла и говорит:

-- Ставь на меня, мужик. Я хоть и старенькая, но раньше чемпионкой была, и сегодня в отличной форме! На меня больше никто не ставит, выиграю забег – сорвешь большой куш!

Подумал-подумал мужик и поставил на кобылу все деньги. Начинается заезд, кобыла дергается с места, делает несколько шагов, падает на землю и больше не поднимается. Мужик подходит к ней после забега с угрожающим видом. Кобыла смотрит на него виновато и шепелявит:

-- Ну не шмогла я, не шмогла...

In [10]:
# семплирование
# Чтобы на каждом шаге выбирать случайное слово с учетом распределения, нужно просто поменять do_sample на True. Так результат будет меняться каждый раз

out = model.generate(input_ids, do_sample=True,  
                     top_k=0,  # про это параметр ниже
                     max_length=50)


generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


За окном дождь. Холодный и противный. Хочется взять зонтик и сбежать вне стен ВУЗа, на улицу.
Рекорд покупок Россияне купили в интернет-магазине 603 392 товара с общей стоимостью 1 906 515 8


Охренеть. Вот это да. Сбер и тут считает деньги. А я чужие не считаю и делать этого не желаю. Дальше. Будем контролировать энтропию или как там было в рике и морти? Выбор может быть слишком случайным, так как иногда будут выбираться совсем маловероятные слова. Есть сразу несколько параметров, которые помогают контролировать случайность.

# top_k
Этот параметр ограничивает количество слов, из которых мы семплируем. 10 - означает, что мы выбирает только из 10 самых вероятных слов. В ячейке выше мы поставили top_k = 0, потому что по умолчанию он стоит 50, а нам нужно было попробовать без него.

Чем больше top_k тем более случайный результат мы получим, но слишком низкий top_k может плохо сказаться на разнообразности.

In [11]:
for top_k in [1,3,10, 30, 100]:

    out = model.generate(input_ids, do_sample=True,  
                     top_k=top_k,  # про это параметр ниже
                     max_length=50)


    generated_text = list(map(tokenizer.decode, out))[0]
    print("### text with top_k - ", top_k)
    print(generated_text)
    print()

### text with top_k -  1
За окном дождь. Холодный и противный. Хочется плакать.

— Я не могу, — говорит она. — Я не могу.

— Почему?

— Потому что я не могу.



### text with top_k -  3
За окном дождь. Холодный и противный. Хочется спать.

Я не могу спать.

Я не могу спать.

Я не могу спать.

И тут меня осенило: я же не могу спать.



### text with top_k -  10
За окном дождь. Холодный и противный. Хочется в тепло и уют.

А ещё сегодня я поняла что у меня очень хороший вкус... Я бы сказала что я его никогда не утрачивала. И не потеряю его никогда.<s>

### text with top_k -  30
За окном дождь. Холодный и противный. Хочется в плед. Плевать, что за окном дождь.

На работе я сижу в теплом кресле и пью горячий чай, а у меня на столе лежит книжка «Сделано у нас»

### text with top_k -  100
За окном дождь. Холодный и противный. Хочется залезть поглубже в одеяло и отогреться.
В общем, я решила, что мне пора брать отпуск, пусть и не очень большой. По крайней мере целых три недели я имею полное



Мой фаворит -- 3. Я не могу спать. Я не могу спать. Я не могу спать.

![sleep](./icantsleep.jpg)

А вообще прикольные и остальные варианты. Можно пошаманить ими после...

# Сэмплирование с Температурой
Еще случайность можно контролировать с помощью параметра, который называется температура. Температура изменяет распределение - при низком значении температуры вероятности переносятся от низких значений к высоким (распределение заостряется), а при высоком - вероятности переносятся от высоких значений к низким (распределение сглаживается).

Это часом не про температуру по палате в среднем?

Нулевая температура означает, что мы на каждом шаге просто выбираем по argmax(), а очень большая температура будет приводить к полному рандому. Под конкретную задачу температуру нужно подбирать отдельно, можно начать с 0 и постепенно увеличивать, смотря на получаемое разнобразие.

(температурой это называется потому что формула взята из физических уравнений, где этот параметр действительно отвечает за температуру)

In [14]:
for temp in [0.001, 0.1, 0.2, 0.5, 0.7, 1., 1.5, 2., 3., 4., 5.]:

    out = model.generate(input_ids, do_sample=True,  
                     top_k=0, 
                     temperature=temp,
                     max_length=50)


    generated_text = list(map(tokenizer.decode, out))[0]
    print("### text with temp - ", temp)
    print(generated_text)
    print()

### text with temp -  0.001
За окном дождь. Холодный и противный. Хочется плакать.

— Я не могучавствовать, — говорит она. — Я не могу.

— Я тоже не могу, — говорит он. —

### text with temp -  0.1
За окном дождь. Холодный и противный. Хочется плакать.

— Я не могу, — говорит она. — Я не могу.

— Почему?

— Потому что я не могу.



### text with temp -  0.2
За окном дождь. Холодный и противный. Хочется есть.

— У меня есть кое-что, — говорит он. — Я могу вас угостить.

— Нет, спасибо, — говорю я.

### text with temp -  0.5
За окном дождь. Холодный и противный. Хочется плакать.

Смотрю на часы. Без десяти четыре.

Включаю компьютер. Нахожу сайт «В контакте».

Смотрю на часы. Без десяти четыре

### text with temp -  0.7
За окном дождь. Холодный и противный. Хочется подождать. Но ожидание стоит того.

— Когда ты успел жениться?

— Недавно.

— Как это не раньше?

— Очень трудно объяснить

### text with temp -  1.0
За окном дождь. Холодный и противный. Хочется под одеяло и не бежать на рабо

Температуру можно сопоставить с шизойдными мыслями в голове, когда человеку плохо во время болезни. Совместим. 

In [15]:
out = model.generate(input_ids, 
                     do_sample=True,
                     temperature=0.9,
                     top_k=10,
                     max_length=50,
                    )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


За окном дождь. Холодный и противный. Хочется есть. Но не хочется идти домой.
- А как же ты? - спросил я.
- А я пойду домой.
- Почему?
- Мне надоело.
- Почему?


Диалоги из евангелиона.

# Beam Search
У подходов выше есть недостаток - на каждом шаге выбирается только 1 слово и этот выбор нельзя изменить, поэтому 1 неверно выбраное слово можно испортить весь текст и это сложно проконтролировать температурой и топ-к.

С этим может помочь beam-search (поиск пучком). Напомню, что в нем по сути происходит одновременная генерация нескольких текстов параллельно и в конце выбирается текст с наибольшей общей вероятностью. Генерировать все возможные варианты невозможно технически (потому что количество вариаций растет очень быстро), поэтому в beam search варианты отсеиваются на каждом шаге так, чтобы количество текущих вариантов было не больше N. Этот параметр N (размер пучка, beam size) настраивается, но поставить его слишком большим не получится, т.к. опять же будет слишком много комбинаций и это увеличит время генерации.

In [16]:
# beam search уже реализован в hg поэтому нужно только задать параметр num_beams
out = model.generate(input_ids, do_sample=True, num_beams=5, top_k=0, max_length=60)

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text.replace('<s>', ' '))


За окном дождь. Холодный и противный. Хочется спрятаться под одеяло, закрыть глаза и не шевелиться.

— Что ты здесь делаешь? — спрашиваю я.

— Жду тебя.

— Зачем?

— Не знаю.

—


Прикольно. Особенно вечное желание прятаться.

# Дообучение GPT

Оказывается, для моего бомжПК есть варианты попроще... Штош попробуем


In [17]:
# возьмем модель поменьше, так как дообучение это обновление весов
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, use_cache=False)

/home/tanaka/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [33]:
input_ids = tokenizer.encode("Дождь шел не переставая с самого утра", return_tensors="pt")
max_length = 100

outputs = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Дождь шел не переставая с самого утра. Дождь лил как из ведра, и он был такой сильный, что казалось, будто он идет по небу.

— Что это? — спросил он.

— Это дождь, — ответила она.

— Что это?

— Это дождь, — повторила она.

— Что это?

— Это дождь, — повторила она.

— Что это?



In [19]:
text = """
«Мой дядя самых честных правил,
Когда не в шутку занемог,
Он уважать себя заставил
И лучше выдумать не мог.
Его пример другим наука;
Но, боже мой, какая скука
С больным сидеть и день и ночь,
Не отходя ни шагу прочь!
Какое низкое коварство
Полуживого забавлять,
Ему подушки поправлять,
Печально подносить лекарство,
Вздыхать и думать про себя:
Когда же черт возьмет тебя!»

Так думал молодой повеса,
Летя в пыли на почтовых,
Всевышней волею Зевеса
Наследник всех своих родных. —
Друзья Людмилы и Руслана!
С героем моего романа
Без предисловий, сей же час
Позвольте познакомить вас:
Онегин, добрый мой приятель,
Родился на брегах Невы,
Где, может быть, родились вы
Или блистали, мой читатель;
Там некогда гулял и я:
Но вреден север для меня.

Служив отлично-благородно,
Долгами жил его отец,
Давал три бала ежегодно
И промотался наконец.
Судьба Евгения хранила:
Сперва Madame за ним ходила,
Потом Monsieur ее сменил;
Ребенок был резов, но мил.
Monsieur l’Abbe€, француз убогой,
Чтоб не измучилось дитя,
Учил его всему шутя,
Не докучал моралью строгой,
Слегка за шалости бранил
И в Летний сад гулять водил.

Когда же юности мятежной
Пришла Евгению пора,
Пора надежд и грусти нежной,
Monsieur прогнали со двора.
Вот мой Онегин на свободе;
Острижен по последней моде;
Как dandy лондонский одет —
И наконец увидел свет.
Он по-французски совершенно
Мог изъясняться и писал;
Легко мазурку танцевал
И кланялся непринужденно;
Чего ж вам больше? Свет решил,
Что он умен и очень мил.

Мы все учились понемногу
Чему-нибудь и как-нибудь,
Так воспитаньем, слава богу,
У нас немудрено блеснуть.
Онегин был, по мненью многих
(Судей решительных и строгих),
Ученый малый, но педант.
Имел он счастливый талант
Без принужденья в разговоре
Коснуться до всего слегка,
С ученым видом знатока
Хранить молчанье в важном споре
И возбуждать улыбку дам
Огнем нежданных эпиграмм.

Латынь из моды вышла ныне:
Так, если правду вам сказать,
Он знал довольно по-латыни,
Чтоб эпиграфы разбирать,
Потолковать об Ювенале,
В конце письма поставить vale,
Да помнил, хоть не без греха,
Из Энеиды два стиха.
Он рыться не имел охоты
В хронологической пыли
Бытописания земли;
Но дней минувших анекдоты,
От Ромула до наших дней,
Хранил он в памяти своей.

Высокой страсти не имея
Для звуков жизни не щадить,
Не мог он ямба от хорея,
Как мы ни бились, отличить.
Бранил Гомера, Феокрита;
Зато читал Адама Смита
И был глубокий эконом,
То есть умел судить о том,
Как государство богатеет,
И чем живет, и почему
Не нужно золота ему,
Когда простой продукт имеет.
Отец понять его не мог
И земли отдавал в залог.

Всего, что знал еще Евгений,
Пересказать мне недосуг;
Но в чем он истинный был гений,
Что знал он тверже всех наук,
Что было для него измлада
И труд, и мука, и отрада,
Что занимало целый день
Его тоскующую лень, —
Была наука страсти нежной,
Которую воспел Назон,
За что страдальцем кончил он
Свой век блестящий и мятежный
В Молдавии, в глуши степей,
Вдали Италии своей.


Как рано мог он лицемерить,
Таить надежду, ревновать,
Разуверять, заставить верить,
Казаться мрачным, изнывать,
Являться гордым и послушным,
Внимательным иль равнодушным!
Как томно был он молчалив,
Как пламенно красноречив,
В сердечных письмах как небрежен!
Одним дыша, одно любя,
Как он умел забыть себя!
Как взор его был быстр и нежен,
Стыдлив и дерзок, а порой
Блистал послушною слезой!

Как он умел казаться новым,
Шутя невинность изумлять,
Пугать отчаяньем готовым,
Приятной лестью забавлять,
Ловить минуту умиленья,
Невинных лет предубежденья
Умом и страстью побеждать,
Невольной ласки ожидать,
Молить и требовать признанья,
Подслушать сердца первый звук,
Преследовать любовь и вдруг
Добиться тайного свиданья…
И после ей наедине
Давать уроки в тишине!


Как рано мог уж он тревожить
Сердца кокеток записных!
Когда ж хотелось уничтожить
Ему соперников своих,
Как он язвительно злословил!
Какие сети им готовил!
Но вы, блаженные мужья,
С ним оставались вы друзья:
Его ласкал супруг лукавый,
Фобласа давний ученик,
И недоверчивый старик,
И рогоносец величавый,
Всегда довольный сам собой,
Своим обедом и женой.


"""

In [20]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Сохраним обучающие данные в .txt файл 
train_path = 'train_dataset.txt'
with open(train_path, "w") as f:
    f.write(text)

# Создание датасета
train_dataset = TextDataset( tokenizer=tokenizer,file_path=train_path,block_size=64, 
                            overwrite_cache=True)
  
# специальный класс который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

2024-05-11 18:02:29.965051: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-11 18:02:31.449818: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/tanaka/.local/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [21]:
# обучение

Через класс Trainer реализовано все обучение, останется только запустить фит (точнее трейн в случае hg)

In [31]:
!pip install accelerate -U

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments( 
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,  
    gradient_accumulation_steps=16, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: python-apt 2.4.0-elementary9-ubuntu7.1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-apt or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [28]:
trainer.train()

NameError: name 'trainer' is not defined

In [ ]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=0.8,
                        top_k=50,
                        max_length=300,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

In [ ]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=50,
                        max_length=300,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

repetition_penalty
Этот параметр просто штрафует повторы. Чем он выше, тем сильнее штраф.

In [ ]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=50,
                        max_length=300,
                        repetition_penalty=3.5
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

no_repeat_ngram_size
А этот параметр напрямую говорит, что нграммы такого размера не должны повторяться совсем

In [ ]:
деятельность и ум.text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=100,
                        max_length=300,
                        repetition_penalty=3.5,
                        no_repeat_ngram_size=2,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

# А теперь по заданию

Хрен с ним, у меня не встает trainer для гопоты, будем колдовать на том, че есть.

In [37]:
likely_text = "Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели: "
input_ids = tokenizer.encode(likely_text, return_tensors="pt")

In [43]:
out = model.generate(input_ids, do_sample=False, max_length=100)


generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text.rstrip())


Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели: 
1) неумение любить; 
2) неумение любить; 
3) неумение любить.


Да ладно. Чувак, ты думаешь, что это действительно две добродетели?

In [42]:
for top_k in [1,3,10, 30, 100]:

    out = model.generate(input_ids, do_sample=True,  
                     top_k=top_k,  # про это параметр ниже
                     max_length=100)


    generated_text = list(map(tokenizer.decode, out))[0]
    print("### text with top_k - ", top_k)
    print(generated_text.rstrip())
    print()

### text with top_k -  1
Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели: 
1) неумение любить; 
2) неумение любить; 
3) неумение любить.

### text with top_k -  3
Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели: 
1) неприступность и стойкость духа и тела; 
2) неуверенность и трусость, и, наконец, неумение и трусость;

### text with top_k -  10
Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели: 
– Счастье и лень;
– Угрызения совести;
– Потеря аппетита;
– Упорство и упорство. 
Супруг мой, по-видимому, был человеком очень умным, и я, в свою очередь, тоже был умным человеком. 
Но что же касается до моих друзей

### text with top_k -  30
Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели: 
1. Вредность — вот основная причина все

Пошел какой-то лютый треш. Попробуем прогреть "божественность" в боте:

In [47]:
for temp in [0.001, 0.1, 0.2, 0.5, 0.7, 1., 1.5, 2., 3., 4., 5.]:

    out = model.generate(input_ids, do_sample=True,  
                     top_k=0, 
                     temperature=temp,
                     max_length=100)


    generated_text = list(map(tokenizer.decode, out))[0]
    print("### text with temp - ", temp)
    print(generated_text.rstrip())
    print()

### text with temp -  0.001
Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели: 
1) неумение любить; 
2) неумение любить; 
3) неумение любить.

### text with temp -  0.1
Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели: 
- любовь к ближнему и к Богу;
- честолюбие и тщеславие.





















 освободиться от всего этогоседая в своем кресле и посмотреть на себя в зеркало.

### text with temp -  0.2
Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели: 
1. Святость.
2. Святость.

### text with temp -  0.5
Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели: 
- любовь к ближнему и к Богу; 
- честолюбие и корысть; 
- любовь к матери и отцу, 
- смирение и послушание; 
- любовь к ближнему и к Богу, 
- любовь к ближнему и к Богу.

##

Прогрев сработал хайпово, будем повышать градус на 0.9. Объеденим подходы для начала на тестовом примере:

In [48]:
out = model.generate(input_ids, 
                     do_sample=True,
                     temperature=0.9,
                     top_k=10,
                     max_length=50,
                    )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели: 
- нестяжание и воздержание; 
- святость и благочестие.



Комплексно теперь прогреваем и играем в k:

In [49]:
for temp in [0.001, 0.1, 0.2, 0.5, 0.7, 1., 1.5, 2., 3., 4., 5.]:
    for top_k in [1,3,10, 30, 100]:
        out = model.generate(input_ids, do_sample=True,  
                     top_k=top_k, 
                     temperature=temp,
                     max_length=100)


        generated_text = list(map(tokenizer.decode, out))[0]
        print("### text with temp - ", temp, " , k - ", top_k)
        print(generated_text.rstrip())
        print()

### text with temp -  0.001  , k -  1
Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели: 
1) неумение любить; 
2) неумение любить; 
3) неумение любить.

### text with temp -  0.001  , k -  3
Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели: 
1) неумение любить; 
2) неумение любить; 
3) неумение любить.

### text with temp -  0.001  , k -  10
Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели: 
1) неумение любить; 
2) неумение любить; 
3) неумение любить.

### text with temp -  0.001  , k -  30
Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели: 
1) неумение любить; 
2) неумение любить; 
3) неумение любить.

### text with temp -  0.001  , k -  100
Он говорил, что есть только два источника людских пороков: праздность и суеверие, 

### text with temp -  1.5  , k -  1
Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели: 
1) неумение любить; 
2) неумение любить; 
3) неумение любить.

### text with temp -  1.5  , k -  3
Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели: 
1) священный долг и благочестие.
2) праздный долг и благочестие.
3) праздность и суеверие.

И, наконец, — что есть только две добродетели: 
1) праздность,
2) праздность
и благочестие.

### text with temp -  1.5  , k -  10
Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели:  — праздное любопытство и жажда власти. — покой и радость жизни.  — праздный страх Божий и смирение и терпение.  -------- 

  &ndash; и вот как я думаю, что это &ndash; одна из добродетельных добродетелей человека&quot;.&nbsp;

### text with temp -  1.5  , k -  30
Он говорил, что есть толь

### text with temp -  5.0  , k -  30
Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели:  
  «Ты в моих книгах Башкортоск, мне я должен дать ответы». Убегайте
. Он спрашивал, можно жить хорошо, имея три дома и
не быть обманутыми: в Баиргее у его дома и  я мог бы работать много хлеба без огорчит и  смерти» А это были его слова? – Я  думаю

### text with temp -  5.0  , k -  100
Он говорил, что есть только два источника людских пороков: праздность и суеверие, и что есть только две добродетели:  искренный характер  правильных побужд…
  Подраса Ашавык никогда никого ничем неустрашит или очаровет в чем нибудь невинующенькими таймами мыслителей человеческих г е ми в жизнь эту глупешествуем как она бывает прекрасна за всех как говорят то это у меня были 2 брата 5 нег =) 4 бра:



Так, ладно. Поиграли в бога и хватит. Нужен какой-то более нейтральный текст. 

In [50]:
likely_text = "Он был на «ты» со всеми, с кем пил шампанское, а пил он шампанское со всеми"
input_ids = tokenizer.encode(likely_text, return_tensors="pt")

In [54]:
for temp in [0.1, 0.2, 0.5, 0.7, 1., 1.5, 2., 3.]:
    for top_k in [1,3,10, 30, 100]:
        out = model.generate(input_ids, do_sample=True, num_beams=5,  
                     top_k=top_k, 
                     temperature=temp,
                     max_length=100)


        generated_text = list(map(tokenizer.decode, out))[0]
        print("### text with temp - ", temp, " , k - ", top_k)
        print(generated_text.replace('<s>', ' ').rstrip())
        print()


### text with temp -  0.1  , k -  1
Он был на «ты» со всеми, с кем пил шампанское, а пил он шампанское со всеми, кто был с ним.

— Я не пью, — сказал он. — Я не пью.

— Я тоже, — сказал я. — Я тоже не пью.

— Я тоже, — сказал он. — Я тоже не пью.

— Я тоже не пью, — сказал я. — Я

### text with temp -  0.1  , k -  3
Он был на «ты» со всеми, с кем пил шампанское, а пил он шампанское со всеми, кто был на «ты» со всеми, кто был на «ты» со всеми, кто был на «ты» со всеми, кто был на «ты» со всеми, кто был на «ты» со всеми, кто был на «ты» со всеми, кто был на «ты» со всеми, кто был на «ты» со всеми, кто был на «ты» со

### text with temp -  0.1  , k -  10
Он был на «ты» со всеми, с кем пил шампанское, а пил он шампанское со всеми, кто был с ним.

— Я не пью, — сказал он. — Я не пью.

— Я тоже, — сказал я. — Я тоже не пью.

— Я тоже, — сказал он. — Я тоже не пью.

— Я тоже не пью, — сказал я. — Я

### text with temp -  0.1  , k -  30
Он был на «ты» со всеми, с кем пил шампанское, а пил он ш

KeyboardInterrupt: 